In [1]:
pip install opencv-python

In [30]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image

In [31]:
data_dir = './CUB_200_2011'
images_dir = os.path.join(data_dir, 'images')
classes_file = os.path.join(data_dir, 'classes.txt')
image_labels_file = os.path.join(data_dir, 'image_class_labels.txt')
train_test_split_file = os.path.join(data_dir, 'train_test_split.txt')
images_file = os.path.join(data_dir, 'images.txt')

In [32]:
# Load class names
images_df = pd.read_csv(images_file, delim_whitespace=True, header=None, names=['serial_no','image_filename'])
num_classes = len(images_df)

In [33]:
# Load image class labels
classes_df = pd.read_csv(classes_file, delim_whitespace=True, header=None, names=['class_label' , 'class_name' ])

image_labels_df = pd.read_csv(image_labels_file, delim_whitespace=True, header=None, names=['serial_no','class_label' ])


# Load train-test split
train_test_split_df = pd.read_csv(train_test_split_file, delim_whitespace=True, header=None, names=['serial_no', 'is_train'])

# Merge dataframes
Y = images_df.merge(train_test_split_df, on='serial_no')

A = Y.merge(image_labels_df, on='serial_no')


data_df = A.merge(classes_df, on='class_label')


In [34]:
print(data_df)


       serial_no                                     image_filename  is_train  \
0              1  001.Black_footed_Albatross/Black_Footed_Albatr...         0   
1              2  001.Black_footed_Albatross/Black_Footed_Albatr...         1   
2              3  001.Black_footed_Albatross/Black_Footed_Albatr...         0   
3              4  001.Black_footed_Albatross/Black_Footed_Albatr...         1   
4              5  001.Black_footed_Albatross/Black_Footed_Albatr...         1   
...          ...                                                ...       ...   
11783      11784  200.Common_Yellowthroat/Common_Yellowthroat_00...         1   
11784      11785  200.Common_Yellowthroat/Common_Yellowthroat_00...         0   
11785      11786  200.Common_Yellowthroat/Common_Yellowthroat_00...         0   
11786      11787  200.Common_Yellowthroat/Common_Yellowthroat_00...         1   
11787      11788  200.Common_Yellowthroat/Common_Yellowthroat_00...         0   

       class_label         

In [35]:
# Split data into training and validation sets
train_df = data_df[data_df['is_train'] == 1]
test_df = data_df[data_df['is_train'] == 0]

In [44]:
image_path=[]
print(train_df['image_filename'])
for file_name in train_df['image_filename']:
    j=os.path.join(images_dir, f'{file_name}')
    image_path.append(j)
    
label=[]
print(train_df['class_label'])
for class_label in train_df['class_label']:
    j= class_label
    label.append(j)

1        001.Black_footed_Albatross/Black_Footed_Albatr...
3        001.Black_footed_Albatross/Black_Footed_Albatr...
4        001.Black_footed_Albatross/Black_Footed_Albatr...
6        001.Black_footed_Albatross/Black_Footed_Albatr...
7        001.Black_footed_Albatross/Black_Footed_Albatr...
                               ...                        
11777    200.Common_Yellowthroat/Common_Yellowthroat_00...
11780    200.Common_Yellowthroat/Common_Yellowthroat_00...
11781    200.Common_Yellowthroat/Common_Yellowthroat_00...
11783    200.Common_Yellowthroat/Common_Yellowthroat_00...
11786    200.Common_Yellowthroat/Common_Yellowthroat_00...
Name: image_filename, Length: 5994, dtype: object
1          1
3          1
4          1
6          1
7          1
        ... 
11777    200
11780    200
11781    200
11783    200
11786    200
Name: class_label, Length: 5994, dtype: int64


In [45]:
print(image_path)
print(label)

['./CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0009_34.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0074_59.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0014_89.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0031_100.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0051_796103.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0010_796097.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0023_796059.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0040_796066.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0089_796069.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0067_170.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0060_796076.

In [42]:
test_paths=[]
print(test_df['image_filename'])
for file_name in test_df['image_filename']:
    j=os.path.join(images_dir, f'{file_name}')
    test_paths.append(j)
    
test_labels=[]
print(test_df['class_label'])
for class_label in test_df['class_label']:
    j= class_label
    test_labels.append(j)

0        001.Black_footed_Albatross/Black_Footed_Albatr...
2        001.Black_footed_Albatross/Black_Footed_Albatr...
5        001.Black_footed_Albatross/Black_Footed_Albatr...
9        001.Black_footed_Albatross/Black_Footed_Albatr...
11       001.Black_footed_Albatross/Black_Footed_Albatr...
                               ...                        
11779    200.Common_Yellowthroat/Common_Yellowthroat_00...
11782    200.Common_Yellowthroat/Common_Yellowthroat_00...
11784    200.Common_Yellowthroat/Common_Yellowthroat_00...
11785    200.Common_Yellowthroat/Common_Yellowthroat_00...
11787    200.Common_Yellowthroat/Common_Yellowthroat_00...
Name: image_filename, Length: 5794, dtype: object
0          1
2          1
5          1
9          1
11         1
        ... 
11779    200
11782    200
11784    200
11785    200
11787    200
Name: class_label, Length: 5794, dtype: int64


In [43]:
print(test_paths)
print(test_labels)

['./CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0046_18.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0002_55.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0085_92.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0025_796057.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0086_796062.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0049_796063.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0006_796065.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0016_796067.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0065_796068.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0042_796071.jpg', './CUB_200_2011\\images\\001.Black_footed_Albatross/Black_Footed_Albatross_0090_7

In [23]:
# Define data pipeline
def preprocess_image(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = img / 255.0  # Normalize to [0,1]
    return img, label

batch_size = 32

In [46]:
train_dataset = tf.data.Dataset.from_tensor_slices((image_path, label))
train_dataset = train_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=len(image_path)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_dataset = test_dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [51]:


def create_model(num_classes):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False  # Freeze the base model

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])

    return model

num_classes = 201  # Number of bird species
model = create_model(num_classes)
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ ?                      │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_3      │ ?                      │   0 (unbuilt) │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [52]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_dataset,
                    validation_data=test_dataset,
                    epochs=10,  # Adjust epochs as needed
                    verbose=1)

Epoch 1/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 218s 1s/step - accuracy: 0.1200 - loss: 4.5491 - val_accuracy: 0.3792 - val_loss: 2.6064
Epoch 2/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 301s 2s/step - accuracy: 0.6009 - loss: 1.7964 - val_accuracy: 0.4586 - val_loss: 2.1545
Epoch 3/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.7708 - loss: 1.1483 - val_accuracy: 0.4972 - val_loss: 1.9711
Epoch 4/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.8530 - loss: 0.8305 - val_accuracy: 0.5212 - val_loss: 1.8756
Epoch 5/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.9157 - loss: 0.6004 - val_accuracy: 0.5223 - val_loss: 1.8713
Epoch 6/10
188/188 ━━━━━━━━━━━━━━━━━━━━ 216s 1s/step - accuracy: 0.9436 - loss: 0.4587 - val_accuracy: 0.5293 - val_loss: 1.8268
Epoch 7/10
 51/188 ━━━━━━━━━━━━━━━━━━━━ 1:19 577ms/step - accuracy: 0.9678 - loss: 0.3500

KeyboardInterrupt: 

In [49]:
import matplotlib.pyplot as plt

def plot_metrics(history):
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

plot_metrics(history)

NameError: name 'history' is not defined